In [1]:
import os

In [2]:
%pwd

'c:\\Users\\amanp\\Documents\\Mlops-Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\amanp\\Documents\\Mlops-Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
import dagshub
dagshub.init(repo_owner='amanpatel.itofficial', repo_name='Mlops-Kidney-Disease-Classification-Deep-Learning-Project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as amanpatel.itofficial

Initialized MLflow to track repo "amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project"

Repository amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project initialized!

🏃 View run blushing-fly-797 at: https://dagshub.com/amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/defab18bb0e343a3931ebb2aa00e7ea4
🧪 View experiment at: https://dagshub.com/amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0


In [6]:
import tensorflow as tf

In [7]:
import mlflow


In [8]:
import os

# List what's in 'artifacts'
print("artifacts/:", os.listdir("artifacts") if os.path.exists("artifacts") else "Not found")

# List what's in 'artifacts/training'
print("artifacts/training/:", os.listdir("artifacts/training") if os.path.exists("artifacts/training") else "Not found")



artifacts/: ['data_ingestion', 'prepare_base_model', 'training']
artifacts/training/: ['model.h5']


In [9]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)   # ✅ fixed bug
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        # ✅ Only tracking URI (no registry)
        mlflow.set_tracking_uri(self.config.mlflow_uri)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # ✅ Always log model as artifact (works on DagsHub)
            mlflow.keras.log_model(self.model, "model")


In [19]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-08-18 01:05:10,980: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-18 01:05:10,983: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-18 01:05:10,984: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 18s 2s/step - loss: 13.1135 - accuracy: 0.5468
[2025-08-18 01:05:29,869: INFO: common: json file saved at: scores.json]


2025/08/18 01:05:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run treasured-snake-922 at: https://dagshub.com/amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0/runs/f9fef5b527ea4818bf8be7b90e3e64d7
🧪 View experiment at: https://dagshub.com/amanpatel.itofficial/Mlops-Kidney-Disease-Classification-Deep-Learning-Project.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}